# Import and installation of Libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
import glob
import os
import time
from pathlib import Path

In [3]:
#pd.set_option('display.max_rows', 500) #This has helped me to see the data a bit more in order to see if my coding was effective. For pdf creation purpose, I have disabled this. 
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

# Read CSV file with all sensors Tags of interest

In [15]:
sensors=pd.read_csv('sensors.csv', header = None)

In [10]:
#sensors_colnames.columns = ["Sensors", "Condition"]
#sensors_colnames

In [11]:
#list_true = sensors_colnames[sensors_colnames['Condition']==True]['Sensors'].tolist()
#list_true

# Main part of the code to read, process, append all the indivual csv file into a set of dataframe (one DF for each sensor at this stage)

In [16]:
start = time.time() #used to calculate processing time

file_path = Path("Raw_furnace_data/") #file path

d = {} #creation of empty dictionary to store all DFs
for i in sensors:
    name = i
    d[name] = pd.DataFrame() #create empty DF within the dictionary
    d[name]= pd.concat([pd.read_csv(j,sep=';',on_bad_lines='skip') for  j in file_path.glob(str(name)+'_[0-9]*.csv')], ignore_index= True) #issue with PID_ZONE_2_OUT fixed with _[0-9], skip bad lines needed as a very few rows are corrupted
    d[name].drop('Name', axis=1, inplace=True) # drop the first column which contains sensors name
    d[name].rename(columns={'Value': str(name)}, inplace=True) #rename the column value with the name of sensor
    d[name]['Date'] =  pd.to_datetime(d[name]['Date'], format='%d/%m/%Y %H:%M:%S', errors='coerce') #convert the date column to a python datetime format
    d[name].set_index('Date', inplace=True) #set the newly formated date column as the index
    d[name] = d[name].sort_index() # sort by index as it was noticed that some csv files contains some data from last year
    d[name]=d[name].loc[d[name].index.notnull()] #
    print (name)
    print (d[name].info())
    print (d[name].head())
    
end = time.time()
print(end - start)

ValueError: No objects to concatenate

In [ ]:
#define start date and end date (we notice issue in the orginal csvs with csv containing data from previous years)
#keep data between these two dates
# this could have been included in previous operation but I have decided to separate it for clarity

start_date='2022-09-06 09:00:00'
end_date='2022-10-14 09:00:00'
start = time.time()
for i in list_true:
    name = i
    d[name] = d[name].loc[start_date:end_date]
    print (name)
    print (d[name].info())
    print (d[name])
    
end = time.time()
print(end - start)

# creating a list of DFs from the dictionary

In [ ]:
#make a combined list with all sensor names
#the issue is that individuals have slightly different lengths

DF_list= list()
for i in list_true:
    name = i
    DF_list.append(d[name])

DF_list

# reducing dataset size using a formatted index

In [ ]:
#define sampling frequency
#create empty df from start date and end date with correct sampling frequency

sampling_frequency='10S'
df_Date_filter = pd.DataFrame({'Date' : pd.date_range(start_date, end_date, freq=sampling_frequency)})
df_Date_filter

In [ ]:
#create a copy of the dictionary for the reduced data (between two dates)
#merge with the reference df with defined sampling frequency
#interpolate data for missing timestamp
#there is a small issue as it creates duplicated index

start = time.time()
import functools as ft
d_red = d.copy()
for i in list_true:
    name = i
    d_red[name] = ft.reduce(lambda  left,right: pd.merge(left,right,on=['Date'],how='outer'), [df_Date_filter, d_red[name]])
    d_red[name].set_index('Date', inplace=True)
    d_red[name] = d_red[name].sort_index()
    d_red[name] = d_red[name].interpolate()
    print (name)
    print (d_red[name].info())
    print (d_red[name])
    
end = time.time()
print(end - start)

In [ ]:
# this step remove the duplicated indexes
#the final DFs have all the same length

start = time.time()
for i in list_true:
    name = i
    d_red[name] = ft.reduce(lambda  left,right: pd.merge(left,right,on=['Date'],how='inner'), [df_Date_filter, d_red[name]])
    d_red[name].set_index('Date', inplace=True)
    d_red[name] = d_red[name].sort_index()
    d_red[name] = d_red[name][~d_red[name].index.duplicated(keep='first')]
    print (name)
    print (d_red[name].info())
    print (d_red[name])
    
end = time.time()
print(end - start)

In [ ]:
d_red['ROOF_0104_300_09_TC'] #exemple of DF for sensor 'ROOF_0104_300_09_TC'

# creating a list of the reduced DFs from the dictionary

In [ ]:
DF_list_red = list()
for i in list_true:
    name = i
    DF_list_red.append(d_red[name])

DF_list_red

# This is the  step to merge all the individuals DFs into a single dataframe

In [ ]:
start = time.time()
#import functools as ft
df_merged_red=DF_list_red[0].copy()

for i in range(1, len(DF_list_red), 1):
    df_merged_red = df_merged_red.merge(DF_list_red[i],on=['Date'],how='left')
end = time.time()
print(end - start)

In [ ]:
df_merged_red

In [ ]:
df_merged_red.info() #we managed to reduced the 5.71 GB datset to a single DF of 90 MB (99.5% reduction in memory usage)

# this step is to export the resulting merged DF to a pickled file for the following plotting and anlaysis operations

In [ ]:
df_merged_red.to_pickle("C:/AURIK/02b_Training/67 - Data_Science_for_Manufacturing_September2022/DS4M Assignment 2 Dec 22/Pickle/df_merged_red.pkl")